In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=b1b7e7400ba5e65afda3288ae0ce4376b6ae6074252cd34f6fa003f0e526d08f
  Stored in directory: /root/.cache/pip/wheels/9a/b8/0f/f580817231cbf59f6ade9fd132ff60ada1de9f7dc85521f857
Successfully built emoji


In [10]:
import numpy as np
import pandas as pd
import re, string
from bs4 import BeautifulSoup
import pickle
import emoji
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from textblob import Word
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
stop_words = set(stopwords.words('english'))

In [6]:
contraction_map={"ain't":'is not',"aren't":'are not',"can't":'can not','cause':'because',"could've":'could have',"couldn't":'could not',"couldn't've":'could not have',"didn't":'did not',"doesn't":'does not',"don't":'do not',"hadn't":'had not',"hadn't've":'had not have',"hasn't":'has not',"haven't":'have not',"he'd":'he would',"he'd've":'he would have',"he'll":'he will',"he'll've":'he will have',"he's":'he is',"how'd":'how did',"how'd've":'how did have',"how'll":'how will',"how's":'how is',"I'd":'I would',"I'd've":'I would have',"I'll":'I will',"I'll've":'I will have',"I'm":'I am',"I've":'I have',"i'd":'i would',"i'd've":'i would have',"i'll":'i will',"i'll've":'i will have',"i'm":'i am',"i've":'i have',"isn't":'is not',"it'd":'it would',"it'd've":'it would have',"it'll":'it will',"it'll've":'it will have',"it's":'it is',"let's":'let us',"ma'am":'madam',"mayn't":'may not',"might've":'might have',"mightn't":'might not',"mightn't've":'might not have',"must've":'might have',"mustn't":'must not',"mustn't've":'must not have',"needn't":'need not',"needn't've":'need not have',"o'clock":'of the clock',"oughtn't":'ought not',"oughtn't've":'ought not have',"shan't":'shall not',"shall'n't":'shall not',"shan't've":'shall not have',"she'd":'she would',"she'd've":'she would have',"she'll":'she will',"she'll've":'she will have',"she's":'she is',"should've":'should have',"shouldn't":'should not',"shouldn't've":'should not have',"so've":'so have',"so's":'so as',"that'd":'that would',"that'd've":'that would have',"that's":'that is',"there'd":'there would',"there'd've":'there would have',"there's":'there is',"they'd":'they would',"they'd've":'they would have',"they'll":'they will',"they'll've":'they will have',"they're":'they are',"they've":'they have',"to've":'to have',"wasn't":'was not',"we'd":'we would',"we'd've":'we would have',"we'll":'we will',"we'll've":'we will have',"we're":'we are',"weren't":'were not',"what'll":'what will',"what'll've":'what will have',"what're":'what are',"what's":'what is',"what've":'what have',"when's":'when is',"when've":'when have',"where'd":'where did',"where's":'where is',"where've":'where have',"who'll":'who will',"who'll've":'who will have',"who's":'who is',"who've":'who have',"why's":'why is',"why've":'why have',"will've":'will have',"won't":'will not',"will't've":'will not have',"would've":'would have',"would't":'would not',"would't've":'would not have',"y'all":'you all',"y'all'd":'you all would',"y'all'd've":'you all would have',"y'all're":'you all are',"y'all've":'you have all',"you'd":'you would',"you'd've":'you would have',"you'll":'you will',"you'll've":'you will have',"you're":'you are',"you've":'you have'}

In [7]:
def expand_contractions(sent, mapping):
    pattern = re.compile('({})'.format('|'.join(mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    def expand_map(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expansion = mapping.get(match) if mapping.get(match) else mapping.get(match.lower())
        expansion = first_char + expansion[1:]
        return expansion
    expand_sent = pattern.sub(expand_map, sent)
    return expand_sent

def split_hashtag(text):
    words = text.split(" ")
    text = ''
    for word in words:
        if(len(word)>0):
            if word[0]=='#':
                tmp=''
                pre=''
                for ch in word:
                    if ch!='#':
                        if (ch<='Z' and ch>='A') and not(pre<='Z' and pre>='A'):
                            tmp+=' '
                        tmp+=ch
                    pre=ch
                word = tmp
        text=text+' '+word
    return text

def replace_emoji(text):
    text = emoji.demojize(text)
    return text

def alpha_only(text):
    tmp=''
    for ch in text:
        if ch>='a' and ch<='z':
            tmp+=ch
        else:
            tmp+=' '
    return tmp

def text_cleaning(text):
    text = BeautifulSoup(text).get_text()
    text = re.sub(r"(?:https?\://)\S+", "", text)
    text=re.sub(r"(?:\@)\S+","user",text)
    text = re.sub('\n','',text)
    text = re.sub('\t','',text)
    text=split_hashtag(text)
    text=replace_emoji(text)
    text=text.lower()
    text = text.replace("’","'")
    text=expand_contractions(text,contraction_map)
    text=alpha_only(text)
    text = ' '.join(text.split())
    return text;

In [8]:
def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

def df_cleaner(df):
    df['text'] = df['text'].apply(text_cleaning)
    df['text'] = df['text'].apply(lambda x: x.split())
    df['text'] = df['text'].apply(remove_stopwords)
    df['text'] = df['text'].apply(lambda x: ' '.join(x))
    return df

def pred_model(model, X):
    y = model.predict(X)
    return y

In [9]:
def get_pred(X):
    LR_pred = pred_model(LR, X)
    MNB_pred = pred_model(MNB,X)
    SGD_pred = pred_model(SGD, X)
    return LR_pred, MNB_pred, SGD_pred
    

In [26]:
src_path="/content/drive/MyDrive/Research/SentimentAnalysis/ExternalDatasets/"

file = open(src_path+'vectoriser.pickle','rb')
vectorizer = pickle.load(file)
file.close()

file = open(src_path+'LR.pickle','rb')
LR = pickle.load(file)
file.close()

file = open(src_path+'MNB.pickle','rb')
MNB = pickle.load(file)
file.close()

file = open(src_path+'BAGG.pickle','rb')
SGD = pickle.load(file)
file.close()

In [14]:
df = pd.read_csv("/content/drive/MyDrive/Research/SentimentAnalysis/MargedTweets/labeled-tweets-all.csv")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1611913 entries, 0 to 1611912
Data columns (total 8 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   id            1611901 non-null  float64
 1   rawContent    1611913 non-null  object 
 2   replyCount    1611901 non-null  float64
 3   retweetCount  1611901 non-null  float64
 4   likeCount     1611901 non-null  float64
 5   quoteCount    1611901 non-null  float64
 6   hashtags      355958 non-null   object 
 7   label         1611913 non-null  object 
dtypes: float64(5), object(3)
memory usage: 98.4+ MB


In [18]:
df_exp = df[['rawContent']]
df_exp=df_exp.rename(columns={"rawContent": "text"})

In [20]:
df_exp.text[0]

'Due to Covid ....#Ukraine #Russia war will be held online.\nThank you'

In [21]:
df_exp=df_cleaner(df_exp)

In [22]:
df_exp.text[0]

'due covid ukraine russia war held online thank'

In [27]:
X = vectorizer.transform(df_exp['text'])

In [28]:
LR_pred, MNB_pred, SGD_pred = get_pred(X)

In [29]:
LR_pred

array([2, 2, 0, ..., 0, 2, 2])

In [41]:
def ensamble_pred(LR_pred, MNB_pred, SGD_pred):
    y_pred = []
    limit = len(LR_pred)
    for i in range(limit):
        neg_ct = 0
        pos_ct = 0
        if LR_pred[i]==0:
            neg_ct+=1
        else:
            pos_ct+=1

        if MNB_pred[i]==0:
            neg_ct+=1
        else:
            pos_ct+=1

        if SGD_pred[i]==0:
            neg_ct+=1
        else:
            pos_ct+=1

        if neg_ct > pos_ct:
            y_pred.append('negative')
        else:
            y_pred.append('positive')

    return y_pred

In [42]:
y_pred=ensamble_pred(LR_pred, MNB_pred, SGD_pred)

In [43]:
df_exp['label']=y_pred

In [44]:
df_exp.text[5]

'stand defense ukraine rights sovereignty freedom victim kuwait aggressor saddam hussein iraq went war aggressor powerful formidable russia world resolves useless sanctions'

In [45]:
df_exp.label[5]

'negative'

In [46]:
df['label_alt']=y_pred

In [47]:
df

,id,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,label,label_alt
0,1.496998e+18,Due to Covid ....#Ukraine #Russia war will be ...,0.0,0.0,1.0,0.0,"['Ukraine', 'Russia']",negative,positive
1,1.496998e+18,Shout out to those brave enough in #Russia to ...,0.0,6.0,6.0,0.0,"['Russia', 'Ukraine', 'Ethiopia', 'Tigray', 'S...",positive,positive
2,1.496998e+18,"In Moscow Russia, 600 people protesting agains...",1.0,0.0,9.0,0.0,"['RussiaUkraineConflict', 'Kharkiv', 'Kyiv', '...",negative,negative
3,1.496998e+18,We attempted to warn the world with this paint...,0.0,1.0,2.0,0.0,"['helmort', 'Russia', 'ukraine', 'RussiaUkrain...",negative,negative
4,1.496998e+18,Follow for more news about this conflict betwe...,0.0,0.0,4.0,0.0,NaN,neutral,negative
...,...,...,...,...,...,...,...,...,...
1611908,1.586508e+18,Ukraine conflict – day 248: Russia ends draft ...,0.0,0.0,0.0,0.0,NaN,negative,negative
1611909,1.586508e+18,@JeffMil00684249 @EtsiSusi My Ukrainian &amp; ...,0.0,0.0,1.0,0.0,"['WhiteNationalists', 'TrumpCult']",positive,positive
1611910,1.586508e+18,The world used Russia to get grain from Ukrain...,3.0,0.0,1.0,0.0,NaN,negative,negative
1611911,1.586508e+18,@Bigtickler2 @myrodovh @Dariusz06239715 @RUnkr...,1.0,0.0,0.0,0.0,NaN,negative,positive


In [48]:
df.to_csv('/content/drive/MyDrive/Research/SentimentAnalysis/LabeledTweetsBinary/label-tweets-all-with-alt.csv',index=False)

In [49]:
df = df[:500]

In [50]:
df

,id,rawContent,replyCount,retweetCount,likeCount,quoteCount,hashtags,label,label_alt
0,1.496998e+18,Due to Covid ....#Ukraine #Russia war will be ...,0.0,0.0,1.0,0.0,"['Ukraine', 'Russia']",negative,positive
1,1.496998e+18,Shout out to those brave enough in #Russia to ...,0.0,6.0,6.0,0.0,"['Russia', 'Ukraine', 'Ethiopia', 'Tigray', 'S...",positive,positive
2,1.496998e+18,"In Moscow Russia, 600 people protesting agains...",1.0,0.0,9.0,0.0,"['RussiaUkraineConflict', 'Kharkiv', 'Kyiv', '...",negative,negative
3,1.496998e+18,We attempted to warn the world with this paint...,0.0,1.0,2.0,0.0,"['helmort', 'Russia', 'ukraine', 'RussiaUkrain...",negative,negative
4,1.496998e+18,Follow for more news about this conflict betwe...,0.0,0.0,4.0,0.0,NaN,neutral,negative
...,...,...,...,...,...,...,...,...,...
495,1.496994e+18,Russia invades Ukraine on many fronts in 'brut...,0.0,0.0,0.0,0.0,NaN,negative,negative
496,1.496994e+18,@MoonMan11_ @elduce83 @notbrxve @kdb17fan @Ukr...,0.0,0.0,1.0,0.0,NaN,negative,positive
497,1.496994e+18,No American soldier needs to die in Ukraine. W...,3.0,1.0,10.0,0.0,NaN,negative,negative
498,1.496994e+18,Russia’s war on Ukraine: European reaction and...,0.0,0.0,0.0,0.0,NaN,neutral,positive


In [51]:
df.to_csv('/content/drive/MyDrive/Research/SentimentAnalysis/LabeledTweetsBinary/label-tweets-all-with-alt-small.csv',index=False)

In [52]:
df=df[['rawContent','label','label_alt']]

In [53]:
df

,rawContent,label,label_alt
0,Due to Covid ....#Ukraine #Russia war will be ...,negative,positive
1,Shout out to those brave enough in #Russia to ...,positive,positive
2,"In Moscow Russia, 600 people protesting agains...",negative,negative
3,We attempted to warn the world with this paint...,negative,negative
4,Follow for more news about this conflict betwe...,neutral,negative
...,...,...,...
495,Russia invades Ukraine on many fronts in 'brut...,negative,negative
496,@MoonMan11_ @elduce83 @notbrxve @kdb17fan @Ukr...,negative,positive
497,No American soldier needs to die in Ukraine. W...,negative,negative
498,Russia’s war on Ukraine: European reaction and...,neutral,positive


In [54]:
df.to_excel('/content/drive/MyDrive/Research/SentimentAnalysis/LabeledTweetsBinary/label-tweets-all-with-alt-small-labels.xlsx',index=False)